# Technical Analysis for Project 1

In [2]:
import warnings
#warnings.filterwarnings('ignore')

In [3]:
import sys
import os
import platform 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import sqlite3
import datetime

In [4]:
import yfinance as yf
import option
import utils

In [5]:
# import your own library
from stock import Stock

# switch to your own version by call TA instead of TA_AP
from TA import *
#from TA import *

In [6]:
opt = option.Option()
# set default settings

# we use relative path here, you can set it with a full path
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlitedb/Equity.db")

# override default option if needed, otherwise use the default settings

In [7]:
print(os.getcwd())

/Users/tu/Desktop/Fall 2023/Computational Methods In Finance/Labs


In [8]:
# check if all the attributes of the option object
print(vars(opt))

{'name': 'standard', 'user': 'tu', 'cli_args': {}, 'environ': 'dev', 'verbose': False, 'force': False, 'start_date': None, 'end_date': None, 'tickers': None, 'port_name': None, 'data_dir': './data', 'output_dir': './data/daily', 'sqlite_db': './data/sqlitedb/Equity.db'}


In [9]:
ticker = 'AAPL'
db_connection = sqlite3.connect(opt.sqlite_db)
stock = Stock(opt, db_connection, ticker)

In [10]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2023, 10, 1)

df = stock.get_daily_hist_price(start_date, end_date)
df.head()

,Ticker,AsOfDate,Open,High,Low,Close,Volume,Dividend,StockSplit
Date,,,,,,,,,
2020-01-02,AAPL,2020-01-02,72.246677,73.309993,71.990608,73.249023,135480400.0,0.0,0.0
2020-01-03,AAPL,2020-01-03,72.468602,73.305105,72.310083,72.536888,146322800.0,0.0,0.0
2020-01-06,AAPL,2020-01-06,71.649169,73.153897,71.395533,73.114876,118387200.0,0.0,0.0
2020-01-07,AAPL,2020-01-07,73.124647,73.383158,72.549096,72.771027,108872000.0,0.0,0.0
2020-01-08,AAPL,2020-01-08,72.471038,74.246476,72.471038,73.941628,132079200.0,0.0,0.0


***Let's do some data visualization***

In [11]:
import plotly.graph_objects as go

In [12]:
fig = go.Figure(data=go.Scatter(x=df.index,y=df['Close'], mode='lines'))
fig.show()

***Now plot the daily price as Candle stick***

https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh

In [13]:
##adding the candle into the map
candlestick = go.Candlestick(
                            x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name = "candlestick"
                            )

fig = go.Figure(data=[candlestick])

fig.show()


### Now load your TA library

In [14]:

import TA as TA

In [15]:
# reload the module if needed
from importlib import reload
reload(TA)

<module 'TA' from '/Users/tu/Desktop/Fall 2023/Computational Methods In Finance/Labs/TA.py'>

### Question 1: Plot the 9, 50, 100, 200 Simple Moving Averages along with the CandleSticks

In [16]:


#calling fetcher's smas
periods = [9, 20, 50, 100, 200]
smas = SimpleMovingAverages(df, periods)
smas.run()
s1 = smas.get_series(9)
s2 = smas.get_series(50)
s3 = smas.get_series(100)
s4 = smas.get_series(200)

#storing the smas into the df
df['s1'] = s1
df['s2'] = s2
df['s3'] = s3
df['s4'] = s4


#plotting the smas
Sma1 = go.Scatter(x=df.index,
                y=df["s1"],
                yaxis="y1",
                name="9 SMA"
                )
Sma2 = go.Scatter(x=df.index,
                y=df["s2"],
                yaxis="y1",
                name="50 SMA"
                )
Sma3 = go.Scatter(x=df.index,
                y=df["s3"],
                yaxis="y1",
                name="100 SMA"
                )
Sma4 = go.Scatter(x=df.index,
                y=df["s4"],
                yaxis="y1",
                name="200 SMA"
                )





fig = go.Figure(data=[candlestick, Sma1, Sma2, Sma3, Sma4])


fig.show()


In [17]:
print(s1.tail())
print(s2.tail())
print(s3.tail())
print(s4.tail())

Date
2023-09-25    175.810001
2023-09-26    175.560001
2023-09-27    174.970000
2023-09-28    174.490000
2023-09-29    173.738890
Name: Close, dtype: float64
Date
2023-09-25    183.001623
2023-09-26    182.566255
2023-09-27    182.105480
2023-09-28    181.622542
2023-09-29    181.189352
Name: Close, dtype: float64
Date
2023-09-25    181.797210
2023-09-26    181.846879
2023-09-27    181.897801
2023-09-28    181.873736
2023-09-29    181.855569
Name: Close, dtype: float64
Date
2023-09-25    165.308277
2023-09-26    165.466371
2023-09-27    165.608301
2023-09-28    165.753971
2023-09-29    165.890640
Name: Close, dtype: float64


***Now plot the SMA and EMA on the same graph***

In [18]:

#ema period not provided. Assumed period = 9

emas = ExponentialMovingAverages(df, periods)
emas.run()
e1 = emas.get_series(9)


#adding ema into df
df['e1'] = e1

#plotting ema
e1 = go.Scatter(x=df.index,
                y=df["e1"],
                name="9 EMA"
                )

#kept the candlestick. Graph has option to show/hide Candle
fig = go.Figure(data=[candlestick, Sma1, Sma2, Sma3, Sma4, e1])
fig.show()

### Question 2: Plot the 50, 100 Exponential Moving Averages along with the Simple Moving Averages

In [19]:

e2 = emas.get_series(50)

df['e2'] = e2

e3 = emas.get_series(100)

df['e3'] = e3

e2 = go.Scatter(x=df.index,
                y=df["e2"],
                
                name="50 EMA"
                )
e3 = go.Scatter(x=df.index,
                y=df["e3"],
                
                name="100 EMA"
                )


fig = go.Figure(data=[Sma1, Sma2, Sma3, Sma4, e2, e3])
fig.show()

### Question 3: Plot RSI with additional 70 and 30 horizontal lines

In [20]:

rsi_indicator = RSI(df)
rsi_indicator.run()

df['rsi'] = rsi_indicator.rsi
df['30'] = 30
df['70'] = 70

line30 = go.Scatter(x=df.index,
                y = df['30'],
                name = "30 line"
                )

line70 = go.Scatter(x=df.index,
                y = df['70'],
                name = "70 line"
                )
r = go.Scatter(x=df.index,
                y = df['rsi'],
                name = "RSI"
                )

fig = go.Figure(data=[ line30, line70, r])
fig.show()


### Question 4: Plot the VWAP along with the 20, 50 Simple Moving Averages and the Candlesticks

In [21]:

Period20 = [20]
smas20 = SimpleMovingAverages(df, Period20)
smas20.run()
s20 = smas20.get_series(20)



df['s20'] = s20
s20 = go.Scatter(x=df.index,
                y=df["s20"],
                yaxis="y1",
                name="20 SMA"
                )


vwap_indicator = VWAP(df)
vwap_indicator.run()
df['vwap'] = vwap_indicator.get_series()

vwap1 = go.Scatter(x=df.index,
                y=df["vwap"],
                yaxis="y1",
                name= "vwap"
                )

#plotting the vwap, sma for 20 and 50
fig = go.Figure(data=[candlestick, s20,Sma2, vwap1])
fig.show(())
